# Jupyter Notebook for the analysis of a microarray experiment in R

### Set the working directory for R and load the packages we will be using

First, make sure that after cloning the directory from git hub you also started R by typing "R" in the terminal window. Since this jupyternotebook is written in R, not bash, you can't use the bash commands you learnt before but have to use R commands. We provided you with a few commands that are useful for this practical. 

First, find out which working directory you are in. Use the provided list of R commands for help. If you're not in "/home/hX/USERNAME/gapractical", you need to set your working directory.

In R, we now need to use three libraries to analyze the microarray. These are limma, statmod and gplots. 

### To do: 
Use the provided list of R commands for help to enter the respective code in the box below. Do not worry if a pink error message field pops up here.  

### The files we will analyze are: 

#### Asurgen_desc.txt: 
A file with descriptions of the samples on the chip. These are four patients from each of which was measured the gene expression in a sample of tumour tissue and a sample of adjacent, healthy tissue. 
#### TumorAdjacent-probe-raw.txt: 
Contains the detection p-values for each probe as well as the intensity of fluorescence obtained by binding of the Oligonucleotide probe on the microchip against the cDNA for each gene and each sample. This file has around 3.2MB. If the experiment had sequenced all cellular mRNA (RNAseq), the file would likely be larger than 1-3GB. 
#### TumorAdjacent-Controls.txt: 
The control probes profile that are informative of the background noise. This could be excess fluorescently labeled cDNA, or other impurities on the chip

#### To Do:
In the three fields below, explore the contents of these three files using the provided list of R code: What are the complete file contents of Asurgen_desc.txt? 
Show the variables and first few rows of data within TumorAdjacent-probe-raw.txt and TumorAdjacent-Controls.txt in the other two fields.

### To do in this field: List what the first four variables within TumorAdjacent-probe-raw.txt represent: 

Variable 1: XXXXXXXXX, <br>
Variable 2: XXXXXXXXX, <br>
Variable 3: XXXXXXXXX, <br>
Variable 4: XXXXXXXXX,<br>

### Using limma, we now combine the input files (can take a few mins)
Now that you have had a look at the files with the microarray data, the background control, and the information about the different types of patient samples, we will begin analysing them with the limma package. The first step is to read the three files in together. The code for this is:

        x = read.ilmn(files = "TumorAdjacent-probe-raw.txt", ctrlfiles = "TumorAdjacent-Controls.txt")
        targets = readTargets("Asurgen_desc.txt")
        
The second step is to normalize the expression data (remove variation from background noise, which is available in the control file) using the function neqc.

        y = neqc(x)

### To do
in the fields below, 
1. Obtain more information about the limma package using the provided list of R code.
2. Obtain more information about the function neqc using the provided list of R code. 
3. Read in the files and use neqc to normalize the expression values. Don't worry if not much happens when running this field - we don't expect to see any outputs just yet.

### Do the different "replicates" (here these are the four different patients) have similar gene expression values, or are there differences between patients?

<blockquote>Background:<br> 
Following Smyth, G.K., Michaud, J. and Scott, H.S., 2005. Use of within-array replicate spots for assessing differential expression in microarray experiments. Bioinformatics, 21(9), pp.2067-2075.<br>
We have four patient samples, from each the tumor and from the adjacent tissue. We want to know which genes are differently expressed in this type of tumor, irrespectively of which patient the samples came from. The intuitive way of doing this would be to compute the average expression value per probe among all four patients per tissue type (tumor/adjacent) and then compare them between tumor and adjacent sample. However, this would reduce our sample size to one and we couldn't do statistics anymore! The duplicateCorrelation method instead computes linear regressions between the expression values of all genes between tumor and adjacent tissue samples, and then compares values for this regression for each gene, between all four patients. If all four patients are similar, the variance among these four separate correlations will be small for each genes.Some genes will have a large variance, which means that we have to be careful when comparing these expression values between tumor and adjacent tissue. The method greatly improves the precision with which the genewise variances are estimated and thereby improves inference methods designed to identify differentially expressed genes</blockquote> 

### To do: 
The code for this procedure is as follows: 
1.First we transform the variable FactorValue (in the file <i>Asurgen_desc.txt</i>, which we transformed into object <i>targets</i>) into a factor (which is a variable with defined levels)and specify the regression matrix (type help(model.matrix) if you want to learn more. 

    ct = factor(targets$FactorValue)
    design = model.matrix(~0+ct)
    colnames(design) = levels(ct)
    
2.Secondly, we perform the duplicateCorrelation operation using <i>y</i>, which are the normalized expression values,<i> design</i>, the model matrix, between the different patients (<i>targets$Patient</i>) <br>   

    dupcor <- duplicateCorrelation(y,design,block=targets$Patient)
    
3.LIMMA assumes that the correlation induced by the blocks is the same for all genes and uses the consensus <i>consensus.correlation</i>. The within-block correlation for each gene is stored on as hyperbolic arctan(correlation) <i>atanh.correlations</i> within the object <i>dupcor</i>. We can plot them as a histogram. <br>

    hist(tanh(dupcor$atanh.correlations))

### Identify genes that are differently expressed between tumor and adjacent tissue 

<blockquote> Background:<br> 
    The results of duplicateCorrelation are now being combined with the "empirical Bayes method" in order to find the genes differently expressed between tumour and adjacent tissue, correcting for any possible variance that exists within the four individuals.<br>

Empirical Bayes method: We will need to shrink the number of genes that are found to be differently expressed by order of evidence for their differential expression using the Empirical Bayes method. Much like duplicateCorrelation, it estimates ’average’ variability over all genes, adjusts high variability genes down and adjusts low variability genes up.<br>

An error probability p-value of p<0.05 essentially means that within 100 statistical tests, the chance to observe a significant effect just by chance or <i>false positives</i> is less than 5%. But for analyzing the microarray, we need to run tests for difference for thousands of probes. That means that we will get a large number of genes that would show up as significantly different between tumor and adjacent tissue, just by chance. So we need another measure to take care of the chance of false discoveries. This is the false discovery rate or <i>FDR</i>.<br> FDR is defined as the expected proportion of Type I errors (false positives) among the rejected hypotheses.

The method was described in this paper:<br> 
Smyth, G. K. (2004). Linear models and empirical Bayes methods for assessing differential expression in microarray experiments. Statistical Applications in Genetics and Molecular Biology 3, Article 3. http://www.statsci.org/smyth/pubs/ebayes.pdf
</blockquote>

### To do:

1.Generate the table of genes that were selected to be differently expressed. 

    fit <- lmFit(y,design,block=targets$Donor,correlation=dupcor$consensus.correlation)
    contrasts = makeContrasts(Tumor-Adjacent, levels = design)
    fit2 = contrasts.fit(fit, contrasts)
    fit3 = eBayes(fit2, trend= TRUE)
    selected = p.adjust(fit3$p.value, method = "fdr") <0.001
    sel = y[selected,]
    matrix = data.matrix(sel$E)
    rownames(matrix) = sel$gene$SYMBOL

2.Look at the table of the results object <i>matrix</i>. What is shown in the rows and columns? 


### V for Vendetta: Next we will be generating a "volcano plot" to visualize the differently expressed genes

<blockquote> Background: <br>
A Volcano plot shows the differences in expression of all genes between the conditions. It will look V-shaped as it shows both the overexpressed and the underexpressed genes. The genes differently expressed will be plotted as log Fold change (x) vs. -log10 p-value for the error probability of differential expression (y). The fold change describes how many-fold a gene changes its expression in the tumor versus the control condition. Those with significant p-value are typically characterized by a different colour than those that are not significantly different </blockquote> 

![A volcano plot to visualize gene expression differences between two samples](volcanoplot.png) 

### To do 

1.First of all, since we are making a plot, we need to load the respective R-package called <i>ggplot2</i>. Look up the respective R command on the help sheet if you don't remember how to load a package.... 

2.Enter the following code to generate the plot. 

    table = data.frame(topTable(fit3, number = 100000, adjust.method = "fdr"))
    table$threshold = as.factor(table$adj.P.Val < 0.001)
    ggplot(data = table, aes(x = logFC, y = -log10(adj.P.Val))) +
	geom_point(alpha = 0.4, size = 1.75, aes(colour = threshold)) +
	theme(legend.position = "none")


### Which genes have the highest expression differences? 

<blockquote>As you can see, there are a lot of differences which, given the nature of cancer being a disease that brings the cell cycle and gene expression out of balance, is not too surprising. In order to find out which genes change their expression with highest fold change in each direction (called "overexpressed" and "underexpressed"), we will generate a heatmap from the top 50 differentially expressed genes. A heatmap is a diagram that shows how genes are over-or underexpressed between the different samples in form of a grid. The sample types are the columns, and the genes are the rows. There is a colour scale which ranges from high to low values The topTable function can help us to do this. The heatmap will show fold changes with respect to the mean fold change of the table. </blockquote>

![A heatmap to visualize gene expression differences between two samples](heatmap.png) 

### To do:
<br>
Enter the code for selecting the most differently expressed genes using <i>topTable</i>.Once the top genes are selected, a heatmap is produced using <i>heatmap.2</i>

    wanted = topTable(fit3, n=50) 
    selected2 = rownames(wanted)
    esetwant = y[selected2,] 
    matrix2 = data.matrix(esetwant$E)
    rownames(matrix2) = wanted$SYMBOL
    library(gplots)
    heatmap.2(matrix2)

### Modify your heatmap 

In the cell below, modify the code to show different colours, trace, and font size  by modifying parameters of the heatmap.2 function. You can add any valid modifiers like this: heatmap.2(matrix2, modifyer=Parameter(NumericalValue)) 

*You can change the colours by adding col = redgreen(), greenred(), bluered(), redblue() or even rainbow(); within the brackets you specify the number of shades of that colour you want
*The cexRow/cexCol modifyer changes the font size of the rows/column labels, eg. cexRow = 0.6  
*trace = “none” gets rid of the density traces on the heatmap. 

Now, generate a new heatmap with a different color scheme, without trace, and with a row fontsize of 0.6 and column fontsize of 1.0.